In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Got airports from table flights
df_airports = pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/airportsFlightsTable.csv")
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   airport_id  376 non-null    int64 
 1   iata_code   376 non-null    object
 2   city        376 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.9+ KB


In [34]:
#Got airports from https://ourairports.com/airports.html
global_airports = pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/worldAirports.csv")

In [35]:
df_global_airports = global_airports[['ident','iata_code','latitude_deg','longitude_deg']]
df_global_airports.rename(columns={'latitude_deg': 'lat','longitude_deg':'lon'}, inplace = True)
df_global_airports

,ident,iata_code,lat,lon
0,EGLL,LHR,51.470600,-0.461941
1,KLAX,LAX,33.942501,-118.407997
2,KORD,ORD,41.978600,-87.904800
3,KJFK,JFK,40.639447,-73.779317
4,KATL,ATL,33.636700,-84.428101
...,...,...,...,...
35484,CMB7,NaN,45.253000,-74.807000
35485,12CN,NaN,39.268306,-122.116689
35486,8CL6,NaN,39.288502,-122.189003
35487,0ID2,NaN,44.442902,-113.773003


In [36]:
df_airports_merged = df_airports.merge(df_global_airports, on = 'iata_code', how = 'inner' )
df_airports_merged = df_airports_merged.drop(columns=['city','ident'])
df_airports_merged['Key'] = df_airports_merged['lat'].astype(str) +'-'+df_airports_merged['lat'].astype(str)
df_airports_merged


,airport_id,iata_code,lat,lon,Key
0,10135,ABE,40.651773,-75.442797,40.651773-40.651773
1,10136,ABI,32.411301,-99.681900,32.4113006592-32.4113006592
2,10140,ABQ,35.040199,-106.609001,35.040199-35.040199
3,10141,ABR,45.449100,-98.421799,45.449100494384766-45.449100494384766
4,10146,ABY,31.532946,-84.196215,31.532946-31.532946
...,...,...,...,...,...
369,15991,YAK,59.508717,-139.660435,59.508717-59.508717
370,16101,YKM,46.568199,-120.543999,46.56819916-46.56819916
371,16133,YNG,41.260700,-80.679100,41.26070023-41.26070023
372,16218,YUM,32.656601,-114.606003,32.656601-32.656601


In [37]:
#Create local file for airports id, iatacode, latitute and longitude
from pathlib import Path  
filepath = Path('/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/Airports_lat_lon.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_airports_merged.to_csv(filepath)  

In [14]:
#Import meteostast library and datetime
from datetime import datetime
from meteostat import Point, Daily

start = datetime(2018, 1, 1) # Change start date
end = datetime(2020, 1, 31) # Change end date

In [15]:
#use weather library meteostat
#Be carefull this library is not availble all the time and generate connection errors
def get_bulk_weather(df):
    weather_dfs=[]
    for i,r in df.iterrows():
        point = Point(r.lat, r.lon)
        data = Daily(point, start, end)
        data = data.fetch()
        data["lat"] = r.lat
        data["lon"] = r.lon
        weather_dfs.append(data)
    weather_final = pd.concat(weather_dfs)
    return weather_final 

df_weather = get_bulk_weather(df_airports_merged)
df_weather

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,lat,lon
2018-01-01,-11.7,-16.0,-7.1,0.0,30.0,276.0,13.0,NaN,1029.3,NaN,40.651773,-75.442797
2018-01-02,-9.4,-14.9,-4.3,0.0,30.0,245.0,17.3,NaN,1029.4,NaN,40.651773,-75.442797
2018-01-03,-8.9,-16.6,-2.7,0.0,0.0,NaN,4.3,NaN,NaN,NaN,40.651773,-75.442797
2018-01-04,-5.0,-9.9,-3.2,2.5,0.0,338.0,24.5,NaN,NaN,NaN,40.651773,-75.442797
2018-01-05,-10.8,-12.7,-8.2,0.0,30.0,285.0,29.9,NaN,1012.7,NaN,40.651773,-75.442797
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
2019-12-26,NaN,-10.6,-4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
2019-12-27,NaN,-11.7,-2.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
2019-12-28,NaN,-11.7,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797


In [16]:
#Create local file to avoid issue with library
from pathlib import Path  
filepath = Path('/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/weatherAirports_2019_2018_Jan2020.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_weather.to_csv(filepath)  

In [22]:
df_weather_f = pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/weatherAirports_2019_2018_Jan2020.csv")  

In [23]:
df_weather_f

,Unnamed: 0,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,lat,lon
0,2018-01-01,-11.7,-16.0,-7.1,0.0,30.0,276.0,13.0,NaN,1029.3,NaN,40.651773,-75.442797
1,2018-01-02,-9.4,-14.9,-4.3,0.0,30.0,245.0,17.3,NaN,1029.4,NaN,40.651773,-75.442797
2,2018-01-03,-8.9,-16.6,-2.7,0.0,0.0,NaN,4.3,NaN,NaN,NaN,40.651773,-75.442797
3,2018-01-04,-5.0,-9.9,-3.2,2.5,0.0,338.0,24.5,NaN,NaN,NaN,40.651773,-75.442797
4,2018-01-05,-10.8,-12.7,-8.2,0.0,30.0,285.0,29.9,NaN,1012.7,NaN,40.651773,-75.442797
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271661,2019-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
271662,2019-12-26,NaN,-10.6,-4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
271663,2019-12-27,NaN,-11.7,-2.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797
271664,2019-12-28,NaN,-11.7,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.258387,-103.748797


In [24]:
df_weather_f = df_weather_f.drop(columns=['tmin','tmax','wdir','pres','wpgt','tsun']) 
df_weather_f.rename(columns={'Unnamed: 0' :'fl_date','tavg':'AvgTemp', 'prcp':'Precipitation', 'snow':'Snow_mm','wspd':'WindSpeed'}, inplace=True)
df_weather_f['Key'] = df_weather_f['lat'].astype(str) +'-'+df_weather_f['lat'].astype(str)


In [27]:
df_weather_f.fillna(0, inplace=True)
#df_weather_f.drop_duplicates(inplace=True)
df_weather_f

,fl_date,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat,lon,Key
0,2018-01-01,-11.7,0.0,30.0,13.0,40.651773,-75.442797,40.651773-40.651773
1,2018-01-02,-9.4,0.0,30.0,17.3,40.651773,-75.442797,40.651773-40.651773
2,2018-01-03,-8.9,0.0,0.0,4.3,40.651773,-75.442797,40.651773-40.651773
3,2018-01-04,-5.0,2.5,0.0,24.5,40.651773,-75.442797,40.651773-40.651773
4,2018-01-05,-10.8,0.0,30.0,29.9,40.651773,-75.442797,40.651773-40.651773
...,...,...,...,...,...,...,...,...
271661,2019-12-25,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271662,2019-12-26,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271663,2019-12-27,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271664,2019-12-28,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387


In [28]:
filepath = Path('/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/processed/weatherAirports_2019_2018_Jan2020_clean.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_weather.to_csv(filepath)  

In [46]:
from add_location_weather import add_location
from add_location_weather import add_weather

In [40]:
weather_airports = pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/processed/weatherAirports_2019_2018_Jan2020_clean.csv")
df_week_one  = pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/Week_one_data_10000.csv")
airports_ids =  pd.read_csv("/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/processed/Airports_lat_lon.csv")


In [42]:
df_week_one

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,day
0,5390,WN,14057,11292,991.0,14057-11292,1
1,2331,WN,15376,11292,639.0,15376-11292,2
2,2321,NK,14843,11618,1608.0,14843-11618,5
3,5496,OO,11109,12266,809.0,11109-12266,3
4,2811,QX,11884,14057,279.0,11884-14057,3
...,...,...,...,...,...,...,...
9995,1447,DL,12953,10397,762.0,12953-10397,2
9996,167,NK,10821,14492,255.0,10821-14492,4
9997,1037,UA,11618,11298,1372.0,11618-11298,7
9998,6130,YX,14122,12953,335.0,14122-12953,3


In [31]:
airports_ids

,Unnamed: 0,airport_id,iata_code,lat,lon
0,0,10135,ABE,40.651773,-75.442797
1,1,10136,ABI,32.411301,-99.681900
2,2,10140,ABQ,35.040199,-106.609001
3,3,10141,ABR,45.449100,-98.421799
4,4,10146,ABY,31.532946,-84.196215
...,...,...,...,...,...
369,369,15991,YAK,59.508717,-139.660435
370,370,16101,YKM,46.568199,-120.543999
371,371,16133,YNG,41.260700,-80.679100
372,372,16218,YUM,32.656601,-114.606003


In [45]:
results = add_location(df_week_one,airports_ids)
results


,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,day,Unnamed: 0,airport_id,iata_code,lat,lon,Key
0,5390,WN,14057,11292,991.0,14057-11292,1,266,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
1,5752,CP,14057,14747,129.0,14057-14747,7,266,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
2,789,UA,14057,11292,991.0,14057-11292,2,266,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
3,1184,AS,14057,14679,933.0,14057-14679,7,266,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
4,1552,AS,14057,14262,873.0,14057-14262,7,266,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,5147,OO,14006,13930,342.0,14006-13930,6,263,14006,PAH,37.060799,-88.773804,37.06079864501953-37.06079864501953
9995,5081,OO,13121,13930,489.0,13121-13930,2,212,13121,LWB,37.858299,-80.399498,37.8582992554-37.8582992554
9996,3845,MQ,11905,11298,140.0,11905-11298,2,137,11905,GGG,32.383999,-94.711502,32.38399887084961-32.38399887084961
9997,4258,OO,12003,14869,463.0,12003-14869,6,148,12003,GTF,47.481998,-111.371002,47.48199844-47.48199844
